In [ ]:
import warnings
warnings.filterwarnings(action='ignore')
%config Computer.use_jedi = False
import requests
import json
import folium
# !pip install pandas # 파이썬 데이터 처리 라이브러리
import pandas as pd
from pandas.io.json import json_normalize # 파이썬 딕셔너리 타입의 데이터를 판다스 데이터프레임으로 변환한다.


전국 또는 특정 지역의 스타벅스 매장 위치 정보를 가져와서 지도 위에 표시하기

requests 모듈로 스타벅스 매장의 위치 정보를 가져와서 딕셔너리 타입으로 변환한다.  
json_normalize() 함수로 json 타입의 데이터가 변환된 딕셔너리를 판다스 데이터프레임으로 변환한다.  
folium 모듈을 사용해서 지도를 표시하고 지도 위에 스타벅스 매장 위치를 마커로 표시한다.  

In [ ]:
# 서울 스타벅스 전 지점
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=78VV3P3AL5'
request = requests.post(targetSite, data={
    'ins_lat': 37.5717888, # 위도
    'ins_lng': 126.9661696, # 경도
    'p_sido_cd': '01', # 시도코드
    'p_gugun_cd': '', # 구군코드
    'in_biz_cd': '', # ? 
    'iend': 2000, # 서버가 응답하는 최대 매장의 개수
    'set_date':  '' # ?
})
starbucks = request.json()
print(type(starbucks))
print(starbucks)

In [ ]:
# pandas.io.json 모듈의 json_normalize() 함수로 json 타입의 데이터가 변환된 딕셔너리를 판다스 데이터프레임으로 변환한다.
# json_normalize(딕셔너리, '데이터프레임으로 변환할 데이터가 할당된 딕셔너리의 key')
starbucks_df = json_normalize(starbucks, 'list')
print(type(starbucks_df)) # <class 'pandas.core.frame.DataFrame'>
print(len(starbucks_df))
print(starbucks_df.columns)

In [ ]:
starbucks_df

In [ ]:
# 데이터프레임으로 변환하니 서울에 있는 스타벅스 지점은 601개고 각 지점을 131개의 컬럼으로 표시하고 있다.
# 작업에 필요한 칼럼 몇 가지를 선택해서 지도에 마커를 표시할 때 사용할 데이터가 저장된 데이터프레임을 만든다.
# s_name => 지점이름
# sido_code => 시도코드
# sido_name => 시도이름
# gugun_code => 구군코드
# gugun_name => 구군이름
# doro_address => 도로명주소
# lat => 위도
# lot => 경도

In [ ]:
# 판다스 데이터프레임에서 특정 컬럼의 데이터를 얻어올 때 얻어올 컬럼이 1개라면 컬럼 이름만 적어도 되지만 얻어올
# 컬럼이 2개 이상일 경우 얻어올 컬럼들을 반드시 리스트로 만들어서 얻어와야 한다.
starbucks_df_map = starbucks_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'doro_address',
                                'lat', 'lot']]
starbucks_df_map

In [ ]:
# dtype 속성으로 데이터프레임을 구성하는 컬럼들의 자료형을 확인할 수 있다.
# 판다스는 문자열 데이터 타입을 object라 부른다.
starbucks_df_map.dtypes

In [ ]:
# info() 메소드를 사용해도 데이터프레임을 구성하는 컬럼들의 자료형을 확인할 수 있다.
starbucks_df_map.info()

In [ ]:
# astype() 메소드로 위도, 경도 데이터 타입을 object에서 float로 변환한다.
starbucks_df_map['lat'] = starbucks_df_map['lat'].astype(float)
starbucks_df_map['lot'] = starbucks_df_map['lot'].astype(float)
starbucks_df_map.info()

In [ ]:
# 종각점
starbucks_df_map[starbucks_df_map['s_name'] == '종각']

In [ ]:
# 종로관찰점
starbucks_df_map[starbucks_df_map['s_name'] == '종로관철']

In [ ]:
# 종로구청점
starbucks_df_map[starbucks_df_map['s_name'] == '종로구청']

In [ ]:
starbucksMap = folium.Map(location=[37.5697987, 126.9837167], zoom_start=16.9)
popup = folium.Popup('JongGak', max_width=300)
folium.Marker(location=[37.569918, 126.984528], popup=popup).add_to(starbucksMap)
popup = folium.Popup('Jongno GwanCheol', max_width=300)
folium.Marker(location=[37.569058,126.986013], popup=popup).add_to(starbucksMap)
popup = folium.Popup('JongnoGu Office', max_width=300)
folium.Marker(location=[37.572662, 126.981138], popup=popup).add_to(starbucksMap)
starbucksMap.save('./starMap1.html')
starbucksMap

In [ ]:
# 서울특별시에 위치한 스타벅스 전 지점의 위치를 지도위에 표시한다.
# 모든 스타벅스 지점의 위도 경도 평균을 중심으로 하자.
# mean() 메소드 : 판다스 데이터프레임에 저장된 데이터의 평균
starbucksMap = folium.Map(location=[starbucks_df_map['lat'].mean(), starbucks_df_map['lot'].mean()], zoom_start=17)
# iterrows() 메소드 : 판다스 데이터프레임에 저장된 인덱스와 데이터를 리턴한다.
for index, data in starbucks_df_map.iterrows():
     # print(index, data) >> 이해 잘못함
    popup = folium.Popup('{}. {}: {}'.format(index + 1, data['s_name'], data['doro_address']), max_width=300)
    folium.Marker(location=[data['lat'], data['lot']], popup=popup).add_to(starbucksMap) # 마커



starbucksMap.save('./starMap2.html')
starbucksMap

In [ ]:
# 서울특별시 종로구 스타벅스 전 지점
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=0B2S7YFEO8'
req = requests.post(targetSite, data={
    'ins_lat': 37.5717888,
    'ins_lng': 126.9661696,
    'p_sido_cd': '',
    'p_gugun_cd': '0118',
    'in_biz_cd': '',
    'iend': 2000,
    'set_date':  ''
})
starbucks = req.json()
starbucks_df = json_normalize(starbucks, 'list')
starbucks_df_map = starbucks_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'doro_address', 'lat', 'lot']]
starbucks_df_map['lat'] = starbucks_df_map['lat'].astype(float)
starbucks_df_map['lot'] = starbucks_df_map['lot'].astype(float)

starbucksMap = folium.Map(location=[starbucks_df_map['lat'].mean(), starbucks_df_map['lot'].mean()], zoom_start=14)
for index, data in starbucks_df_map.iterrows():
    popup = folium.Popup('{}. {}: {}'.format(index + 1, data['s_name'], data['doro_address']), max_width=300)
    folium.Marker(location=[data['lat'], data['lot']], popup=popup).add_to(starbucksMap)
starbucksMap.save('./starMap3.html')
starbucksMap

In [ ]:
# 경기도 스타벅스 전 지점
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=0B2S7YFEO8'
req = requests.post(targetSite, data={
    'ins_lat': 37.5717888,
    'ins_lng': 126.9661696,
    'p_sido_cd': '08',
    'p_gugun_cd': '',
    'in_biz_cd': '',
    'iend': 2000,
    'set_date':  ''
})
starbucks = req.json()
starbucks_df = json_normalize(starbucks, 'list')
starbucks_df_map = starbucks_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'doro_address', 'lat', 'lot']]
starbucks_df_map['lat'] = starbucks_df_map['lat'].astype(float)
starbucks_df_map['lot'] = starbucks_df_map['lot'].astype(float)

starbucksMap = folium.Map(location=[starbucks_df_map['lat'].mean(), starbucks_df_map['lot'].mean()], zoom_start=10)
for index, data in starbucks_df_map.iterrows():
    popup = folium.Popup('{}. {}: {}'.format(index + 1, data['s_name'], data['doro_address']), max_width=300)
    folium.Marker(location=[data['lat'], data['lot']], popup=popup).add_to(starbucksMap)
starbucksMap.save('./starMap3.html')
starbucksMap